In [4]:
import os
import sys

In [5]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import os

output_dir = '../data'
output_path = os.path.join(output_dir, 'df_cleaned.parquet')

try:
    df = pd.read_parquet(output_path)
except FileNotFoundError:
    print("Файл отсутствует. Проверьте этап запуска.")